In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTENC
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import model_selection

import matplotlib.pyplot as plt

from optbinning import MulticlassOptimalBinning

import warnings
warnings.filterwarnings("ignore")

import sklearn
from flaml import AutoML


C:\Users\Lucas\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\Lucas\Anaconda3\lib\site-packages\dask\config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
%%time
train_values = pd.read_csv("train_values.csv")
train_labels = pd.read_csv("train_labels.csv")
test_values = pd.read_csv("test_values.csv")

Wall time: 825 ms


In [6]:
train_labels.damage_grade.unique()

array([3, 2, 1], dtype=int64)

In [25]:
for df in [train_values, test_values]:
    var_nm = 'has_superstructure_adobe_mud'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 0.5
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 1
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 0
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 300

    var_nm = 'has_superstructure_mud_mortar_stone'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 2
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 2
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 800
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 800

    var_nm = 'has_superstructure_stone_flag'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 25
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 115
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 1200
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 1200

    var_nm = 'has_superstructure_cement_mortar_stone'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 20
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 40
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 14000
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 28000

    var_nm = 'has_superstructure_mud_mortar_brick'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 2
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 2
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 800
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 1200

    var_nm = 'has_superstructure_cement_mortar_brick'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 6
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 20
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 4200
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 14000

    var_nm = 'has_superstructure_timber'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 50
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 50
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 13000
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 13000

    var_nm = 'has_superstructure_bamboo'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 40
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 40
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 15000
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 15000

    var_nm = 'has_superstructure_rc_non_engineered'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 25
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 35
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 20000
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 20000

    var_nm = 'has_superstructure_rc_engineered'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 38
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 38
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 32836.6
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 32836.6

    var_nm = 'has_superstructure_bamboo'
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_min'] = 40
    df.loc[df[var_nm] == 1, f'{var_nm}_cs_max'] = 40
    df.loc[df[var_nm] == 1, f'{var_nm}_em_min'] = 15000
    df.loc[df[var_nm] == 1, f'{var_nm}_em_max'] = 15000
    
    df.fillna(0, inplace=True)

In [26]:
# find number of rows and columns
print(
f'train_values shape: {train_values.shape}\n\
train_labels shape: {train_labels.shape}\n\
test_values shape: {test_values.shape}')

train_values shape: (260601, 79)
train_labels shape: (260601, 2)
test_values shape: (86868, 79)


In [76]:
# look at the last 5 rows
train_values.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,has_superstructure_adobe_mud_cs_min,has_superstructure_adobe_mud_cs_max,has_superstructure_adobe_mud_em_min,has_superstructure_adobe_mud_em_max,has_superstructure_mud_mortar_stone_cs_min,has_superstructure_mud_mortar_stone_cs_max,has_superstructure_mud_mortar_stone_em_min,has_superstructure_mud_mortar_stone_em_max,has_superstructure_stone_flag_cs_min,has_superstructure_stone_flag_cs_max,has_superstructure_stone_flag_em_min,has_superstructure_stone_flag_em_max,has_superstructure_cement_mortar_stone_cs_min,has_superstructure_cement_mortar_stone_cs_max,has_superstructure_cement_mortar_stone_em_min,has_superstructure_cement_mortar_stone_em_max,has_superstructure_mud_mortar_brick_cs_min,has_superstructure_mud_mortar_brick_cs_max,has_superstructure_mud_mortar_brick_em_min,has_superstructure_mud_mortar_brick_em_max,has_superstructure_cement_mortar_brick_cs_min,has_superstructure_cement_mortar_brick_cs_max,has_superstructure_cement_mortar_brick_em_min,has_superstructure_cement_mortar_brick_em_max,has_superstructure_timber_cs_min,has_superstructure_timber_cs_max,has_superstructure_timber_em_min,has_superstructure_timber_em_max,has_superstructure_bamboo_cs_min,has_superstructure_bamboo_cs_max,has_superstructure_bamboo_em_min,has_superstructure_bamboo_em_max,has_superstructure_rc_non_engineered_cs_min,has_superstructure_rc_non_engineered_cs_max,has_superstructure_rc_non_engineered_em_min,has_superstructure_rc_non_engineered_em_max,has_superstructure_rc_engineered_cs_min,has_superstructure_rc_engineered_cs_max,has_superstructure_rc_engineered_em_min,has_superstructure_rc_engineered_em_max
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.5,1.0,0.0,300.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,50.0,13000.0,13000.0,40.0,40.0,15000.0,15000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.5,1.0,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
train_labels.tail()

,building_id,damage_grade
260596,688636,2
260597,669485,3
260598,602512,3
260599,151409,2
260600,747594,3


In [29]:
train_values.describe()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,has_superstructure_bamboo_em_min,has_superstructure_bamboo_em_max,has_superstructure_rc_non_engineered_cs_min,has_superstructure_rc_non_engineered_cs_max,has_superstructure_rc_non_engineered_em_min,has_superstructure_rc_non_engineered_em_max,has_superstructure_rc_engineered_cs_min,has_superstructure_rc_engineered_cs_max,has_superstructure_rc_engineered_em_min,has_superstructure_rc_engineered_em_max
count,2.606010e+05,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,...,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
mean,5.256755e+05,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,0.088645,0.761935,...,1275.167785,1275.167785,1.064750,1.490650,851.800262,851.800262,0.602661,0.602661,520.771861,520.771861
std,3.045450e+05,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,0.284231,0.425900,...,4183.483125,4183.483125,5.048283,7.067596,4038.626516,4038.626516,4.747420,4.747420,4102.345493,4102.345493
min,4.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.611900e+05,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.257570e+05,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.897620e+05,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.052934e+06,30.000000,1427.000000,12567.000000,9.000000,995.000000,100.000000,32.000000,1.000000,1.000000,...,15000.000000,15000.000000,25.000000,35.000000,20000.000000,20000.000000,38.000000,38.000000,32836.600000,32836.600000


In [30]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 79 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   building_id                                    260601 non-null  int64  
 1   geo_level_1_id                                 260601 non-null  int64  
 2   geo_level_2_id                                 260601 non-null  int64  
 3   geo_level_3_id                                 260601 non-null  int64  
 4   count_floors_pre_eq                            260601 non-null  int64  
 5   age                                            260601 non-null  int64  
 6   area_percentage                                260601 non-null  int64  
 7   height_percentage                              260601 non-null  int64  
 8   land_surface_condition                         260601 non-null  object 
 9   foundation_type                      

In [31]:
# find non numerical features (categorical)
categ_col = []
for col in train_values.columns:
    if train_values[col].dtypes != 'int64':
        categ_col.append(col)

In [32]:
# function to encode categorical features into numerical features to prepare for model fitting

from sklearn.preprocessing import OneHotEncoder

def prepare_inputs(X_train, X_test):
    categ_col_train = X_train.select_dtypes(include=[object]).copy()
    categ_col_test = X_test.select_dtypes(include=[object]).copy()
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
    ohe.fit(categ_col_train)
    

    categ_col_test.index = X_test.index
    
    enc_train_categ = pd.DataFrame(ohe.transform(categ_col_train))
    enc_train_categ.index = X_train.index
    X_train_enc = pd.concat([enc_train_categ, X_train.select_dtypes(exclude=[object])], axis=1)
    
    print(f'left table: {enc_train_categ.shape}, right table: {X_train.select_dtypes(exclude=[object]).shape}\
merged table: {X_train_enc.shape}')
    
    enc_test_categ = pd.DataFrame(ohe.transform(categ_col_test))
    enc_test_categ.index = X_test.index
    X_test_enc = pd.concat([enc_test_categ, X_test.select_dtypes(exclude=[object])], axis=1)
   
    print(f'left table: {enc_test_categ.shape}, right table: {X_test.select_dtypes(exclude=[object]).shape}\
merged table: {X_test_enc.shape}')
    return X_train_enc, X_test_enc

In [33]:
# possible values and distributions of the label
train_labels.damage_grade.value_counts()

2    148259
3     87218
1     25124
Name: damage_grade, dtype: int64

In [34]:
def binning(variable, X_train, y_train, X_val=None, X_test=None, show_plot=True, verbose=False):
    optb = MulticlassOptimalBinning(name=variable, solver="cp")
    optb.fit(X_train[variable], y_train)

    binning_table = optb.binning_table
    bin_df = binning_table.build()
    
    if show_plot:
        binning_table.plot(add_special=False, add_missing=False)
        
    var_transform_woe = optb.transform(X_train[variable], metric="weighted_mean_woe")
    var_bins = optb.transform(X_train[variable], metric="bins")
    
    if verbose:
        print(optb.status)
        print(optb.splits)
        print(pd.Series(var_bins).value_counts())
        
    X_train_out = X_train.copy()
    X_train_out[f'{variable}_bins'] = var_bins
    
    if X_val is not None:
        X_val_out = X_val.copy()
        X_test_out = X_test.copy()
        for df in [X_val_out, X_test_out]:
            for col in [variable]:
                uniq_v = df[col].unique()
                train_vs = X_train_out[col].unique()
                for col_v in uniq_v:
                    if col_v in train_vs:
                        df.loc[df[col]==col_v, f'{variable}_bins'] = X_train_out.loc[(X_train_out[col]==col_v),f'{variable}_bins'].values[0]
                    else:
                        df.loc[df[col]==col_v, f'{variable}_bins'] = 'never seen'
                            
        return bin_df, X_train_out, X_val_out, X_test_out
    
    return bin_df, X_train_out

In [10]:
for col in train_values.columns:
    if col in categ_col:
        train_values[col].value_counts().plot(kind='bar').set_title(col)
    else:
        plt.hist(train_values[col], bins=30)
        plt.xlabel(col)
        plt.show()

NameError: name 'categ_col' is not defined

In [9]:
train_values.age.value_counts().sort_index()

0      26041
5      33697
10     38896
15     36010
20     32182
25     24366
30     18028
35     10710
40     10559
45      4711
50      7257
55      2033
60      3612
65      1123
70      1975
75       512
80      3055
85       847
90      1085
95       414
100     1364
105       89
110      100
115       21
120      180
125       37
130        9
135        5
140        9
145        3
150      142
155        1
160        6
165        2
170        6
175        5
180        7
185        1
190        3
195        2
200      106
995     1390
Name: age, dtype: int64

In [36]:
def prob_feature(cols, X_train, y_train, X_val=None, X_test=None):
    fea = pd.merge(X_train, y_train, left_index=True, right_index=True)
    X_train_out = X_train.copy()
    target_values = sorted(fea['damage_grade'].unique())
    for col in cols:
        for v in target_values:
            damage = {}
            for i,j in (X_train_out[col].value_counts()).iteritems():
                n = len(fea[fea['damage_grade']==v][fea[col]==i])
                damage[i] = n/j
            lst = []
            for i in X_train_out[col]:
                lst.append(damage.get(i))
            X_train_out[f'{col}_prob{v}'] = lst  
    
    if X_val is not None:
        X_val_out = X_val.copy()
        X_test_out = X_test.copy()
        for df in [X_val_out, X_test_out]:
            for col in cols:
                uniq_v = df[col].unique()
                train_vs = X_train_out[col].unique()
                for col_v in uniq_v:
                    for target_v in target_values:
                        if col_v in train_vs:
                            df.loc[df[col]==col_v, f'{col}_prob{target_v}'] = \
                                X_train_out.loc[(X_train_out[col]==col_v),f'{col}_prob{target_v}'].values[0]
                        else:
                            df.loc[df[col]==col_v, f'{col}_prob{target_v}'] = 0
        return X_train_out, X_val_out, X_test_out
    
    return X_train_out

In [37]:
def geo_weight_encoding(X_train, y_train, X_val=None, X_test=None):
    fea = pd.merge(X_train, y_train.drop(columns='building_id'), left_index=True, right_index=True)

    geo1_grp = fea.groupby('geo_level_1_id').agg({'building_id':'count', 'damage_grade': sum}).reset_index()
    geo2_grp = fea.groupby(['geo_level_1_id', 'geo_level_2_id']).agg({'building_id':'count', 'damage_grade': sum}).reset_index()
    geo3_grp = fea.groupby(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']).agg({'building_id':'count', 'damage_grade': sum}).reset_index()
    
    geo1_grp.rename(columns={'building_id':'cnt_1', 'damage_grade': 'dmg_sum_1'}, inplace=True)
    geo2_grp.rename(columns={'building_id':'cnt_2', 'damage_grade': 'dmg_sum_2'}, inplace=True)
    geo3_grp.rename(columns={'building_id':'cnt_3', 'damage_grade': 'dmg_sum_3'}, inplace=True)
    all_grp = pd.merge(geo3_grp, geo2_grp, on=['geo_level_1_id','geo_level_2_id'])\
                .merge(geo1_grp, on='geo_level_1_id')
    
    all_grp['wdmg_geo32'] = (all_grp['dmg_sum_3'] + all_grp['dmg_sum_2']) / (all_grp['cnt_3'] + (all_grp['cnt_2']))
    all_grp['wdmg_geo321'] = (all_grp['dmg_sum_3'] + all_grp['dmg_sum_2'] + all_grp['dmg_sum_1']) / \
                             (all_grp['cnt_3'] + all_grp['cnt_2'] + all_grp['cnt_1'])

    all_grp.drop(columns=['cnt_3','cnt_2','cnt_1','dmg_sum_3','dmg_sum_2','dmg_sum_1'], inplace=True)
    X_train_out = pd.merge(X_train, all_grp, on=['geo_level_3_id','geo_level_2_id','geo_level_1_id'], how='left')
    X_train_out.sort_values(by='geo_level_3_id', inplace=True)
    if X_val is not None:
        X_val_out = pd.merge(X_val, all_grp, on=['geo_level_3_id','geo_level_2_id','geo_level_1_id'], how='left')
        X_test_out = pd.merge(X_test, all_grp, on=['geo_level_3_id','geo_level_2_id','geo_level_1_id'], how='left')
        return X_train_out, X_val_out, X_test_out
    
    return X_train_out

In [38]:
train_values.isnull().any()

building_id                                    False
geo_level_1_id                                 False
geo_level_2_id                                 False
geo_level_3_id                                 False
count_floors_pre_eq                            False
                                               ...  
has_superstructure_rc_non_engineered_em_max    False
has_superstructure_rc_engineered_cs_min        False
has_superstructure_rc_engineered_cs_max        False
has_superstructure_rc_engineered_em_min        False
has_superstructure_rc_engineered_em_max        False
Length: 79, dtype: bool

In [39]:
train_values.head(2)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_superstructure_bamboo_em_min,has_superstructure_bamboo_em_max,has_superstructure_rc_non_engineered_cs_min,has_superstructure_rc_non_engineered_cs_max,has_superstructure_rc_non_engineered_em_min,has_superstructure_rc_non_engineered_em_max,has_superstructure_rc_engineered_cs_min,has_superstructure_rc_engineered_cs_max,has_superstructure_rc_engineered_em_min,has_superstructure_rc_engineered_em_max
0,802906,6,487,12198,2,30,6,5,t,r,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28830,8,900,2812,2,10,8,7,o,r,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Splitting the Data

In [40]:
cols_to_drop = ['building_id']

In [41]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_values, train_labels['damage_grade'], test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(y_train.value_counts())

(208480, 79)
(208480,)
2    118772
3     69754
1     19954
Name: damage_grade, dtype: int64


In [44]:
%%time
cols = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
X_train, X_test, test_values = prob_feature(cols, X_train, y_train, X_test, test_values)

Wall time: 19min 34s


In [45]:
X_train.to_pickle('./tmp_data/X_train_geo_prob_structure.pkl')
X_test.to_pickle('./tmp_data/X_test_geo_prob_structure.pkl')
test_values.to_pickle('./tmp_data/test_values_geo_prob_structure.pkl')

In [93]:
X_train, X_test, test_values = geo_weight_encoding(X_train, y_train, X_test, test_values)

In [95]:
%%time
_, X_train, X_test, test_values = binning('age', X_train, y_train, X_test, test_values, show_plot=False)

Wall time: 668 ms


In [99]:
%%time
X_train, X_test, test_values = prob_feature(['age_bins'], X_train, y_train, X_test, test_values)

Wall time: 1.01 s


In [46]:
X_train

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_superstructure_rc_engineered_em_max,geo_level_1_id_prob1,geo_level_1_id_prob2,geo_level_1_id_prob3,geo_level_2_id_prob1,geo_level_2_id_prob2,geo_level_2_id_prob3,geo_level_3_id_prob1,geo_level_3_id_prob2,geo_level_3_id_prob3
170377,739427,13,257,8352,3,25,2,5,n,r,...,0.0,0.203597,0.650808,0.145594,0.370130,0.590909,0.038961,0.000000,1.000000,0.000000
184895,17201,6,1076,9202,3,40,10,8,o,r,...,0.0,0.083862,0.666650,0.249488,0.008197,0.540984,0.450820,0.025000,0.650000,0.325000
26740,723805,7,838,10723,2,45,8,5,t,r,...,0.0,0.054088,0.591804,0.354108,0.079137,0.208633,0.712230,0.103448,0.137931,0.758621
198210,891512,7,555,2763,2,30,7,4,t,r,...,0.0,0.054088,0.591804,0.354108,0.006472,0.851133,0.142395,0.000000,0.921569,0.078431
254313,484350,17,682,1039,3,30,6,7,t,r,...,0.0,0.013117,0.179184,0.807699,0.000000,0.048193,0.951807,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,79888,13,909,1626,2,25,6,5,t,r,...,0.0,0.203597,0.650808,0.145594,0.050847,0.796610,0.152542,0.068966,0.620690,0.310345
103694,442764,10,1397,8182,2,5,5,4,n,r,...,0.0,0.054497,0.548533,0.396970,0.078244,0.568702,0.353053,0.023256,0.116279,0.860465
131932,17256,8,1073,5249,3,20,22,7,o,r,...,0.0,0.034573,0.451071,0.514356,0.019608,0.810458,0.169935,0.000000,0.935484,0.064516
146867,496181,10,76,1841,3,10,11,7,t,r,...,0.0,0.054497,0.548533,0.396970,0.037528,0.759382,0.203091,0.107143,0.687500,0.205357


In [21]:
# categ_col.remove('age_bins')
# categ_col.remove('bins')

In [107]:
categ_col = categ_col + ['age_bins']
print(categ_col)
categ_col_index = [X_train.columns.get_loc(c) for c in categ_col]
print(categ_col_index)

['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status', 'age_bins', 'age_bins']
[8, 9, 10, 11, 12, 13, 14, 26, 50, 50]


In [47]:
X_train.drop(columns='building_id',inplace=True)
X_test.drop(columns='building_id',inplace=True)
test_values.drop(columns='building_id',inplace=True)

In [48]:
%%time
# rebalance data

rebalance = 0

if rebalance == 1:
    smote = SMOTENC(categorical_features=categ_col_index, random_state=0)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    print(X_resampled.shape)
    print(y_resampled.shape)
    print(y_resampled.value_counts())
else:
    pass

Wall time: 0 ns


In [56]:
X_train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,has_superstructure_adobe_mud_cs_min,has_superstructure_adobe_mud_cs_max,has_superstructure_adobe_mud_em_min,has_superstructure_adobe_mud_em_max,has_superstructure_mud_mortar_stone_cs_min,has_superstructure_mud_mortar_stone_cs_max,has_superstructure_mud_mortar_stone_em_min,has_superstructure_mud_mortar_stone_em_max,has_superstructure_stone_flag_cs_min,has_superstructure_stone_flag_cs_max,has_superstructure_stone_flag_em_min,has_superstructure_stone_flag_em_max,has_superstructure_cement_mortar_stone_cs_min,has_superstructure_cement_mortar_stone_cs_max,has_superstructure_cement_mortar_stone_em_min,has_superstructure_cement_mortar_stone_em_max,has_superstructure_mud_mortar_brick_cs_min,has_superstructure_mud_mortar_brick_cs_max,has_superstructure_mud_mortar_brick_em_min,has_superstructure_mud_mortar_brick_em_max,has_superstructure_cement_mortar_brick_cs_min,has_superstructure_cement_mortar_brick_cs_max,has_superstructure_cement_mortar_brick_em_min,has_superstructure_cement_mortar_brick_em_max,has_superstructure_timber_cs_min,has_superstructure_timber_cs_max,has_superstructure_timber_em_min,has_superstructure_timber_em_max,has_superstructure_bamboo_cs_min,has_superstructure_bamboo_cs_max,has_superstructure_bamboo_em_min,has_superstructure_bamboo_em_max,has_superstructure_rc_non_engineered_cs_min,has_superstructure_rc_non_engineered_cs_max,has_superstructure_rc_non_engineered_em_min,has_superstructure_rc_non_engineered_em_max,has_superstructure_rc_engineered_cs_min,has_superstructure_rc_engineered_cs_max,has_superstructure_rc_engineered_em_min,has_superstructure_rc_engineered_em_max,geo_level_1_id_prob1,geo_level_1_id_prob2,geo_level_1_id_prob3,geo_level_2_id_prob1,geo_level_2_id_prob2,geo_level_2_id_prob3,geo_level_3_id_prob1,geo_level_3_id_prob2,geo_level_3_id_prob3
170377,13,257,8352,3,25,2,5,n,r,n,f,x,s,q,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.5,1.0,0.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203597,0.650808,0.145594,0.370130,0.590909,0.038961,0.000000,1.000000,0.000000
184895,6,1076,9202,3,40,10,8,o,r,n,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,0,1,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083862,0.666650,0.249488,0.008197,0.540984,0.450820,0.025000,0.650000,0.325000
26740,7,838,10723,2,45,8,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,w,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054088,0.591804,0.354108,0.079137,0.208633,0.712230,0.103448,0.137931,0.758621
198210,7,555,2763,2,30,7,4,t,r,n,v,q,s,d,0,1,0,1,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,20.0,40.0,14000.0,28000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054088,0.591804,0.354108,0.006472,0.85

In [50]:
# use function prepare_inputs to convert categorical features to numerical features
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)

if rebalance == 1:
    X_train_enc_reb, X_test_enc_reb = prepare_inputs(X_resampled, X_test)

left table: (208480, 38), right table: (208480, 79)merged table: (208480, 117)
left table: (52121, 38), right table: (52121, 79)merged table: (52121, 117)


In [51]:
print(X_train_enc.shape)

if rebalance == 1:
    print(X_train_enc_reb.shape)

(208480, 117)


In [52]:
pd.set_option("display.max_columns", None)

In [53]:
X_test_enc = X_test_enc.fillna(y_train.mean())

# XGBoosting

In [72]:
%%time
model = XGBClassifier()
if rebalance == 0:
    model.fit(X_train_enc, y_train)
else:
    print('**rebalanced**')
    model.fit(X_train_enc_reb, y_resampled)

Wall time: 50.1 s


In [73]:
if rebalance == 0:
    y_pred = model.predict(X_test_enc)
else:
    print("**rebalanced**")
    y_pred = model.predict(X_test_enc_reb)
predictions = [round(value) for value in y_pred]

In [74]:
class_names = [str(x) for x in model.classes_]
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

           1       0.67      0.54      0.60      5170
           2       0.75      0.84      0.79     29487
           3       0.75      0.64      0.69     17464

    accuracy                           0.74     52121
   macro avg       0.72      0.67      0.69     52121
weighted avg       0.74      0.74      0.74     52121



In [75]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score = {:.4f}".format(f1_score(y_test, predictions, average='micro')))

Accuracy: 74.03%
F1 Score = 0.7403


# FLAML

In [68]:
automl = AutoML()
automl_settings = {
    "time_budget": 600,  # in seconds
    "metric": 'micro_f1',
    "task": 'classification',
    "log_file_name": "EQ.log",
}

In [69]:
automl.fit(X_train=X_train_enc, y_train=y_train.squeeze(),
           **automl_settings)

[flaml.automl: 12-01 18:03:06] {1926} INFO - task = classification
[flaml.automl: 12-01 18:03:06] {1928} INFO - Data split method: stratified
[flaml.automl: 12-01 18:03:06] {1932} INFO - Evaluation method: holdout
[flaml.automl: 12-01 18:03:07] {1999} INFO - Minimizing error metric: 1-micro_f1
[flaml.automl: 12-01 18:03:07] {2051} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl: 12-01 18:03:07] {2291} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-01 18:03:07] {2405} INFO - Estimated sufficient time budget=12912s. Estimated necessary time budget=298s.
[flaml.automl: 12-01 18:03:07] {2485} INFO -  at 4.1s,	estimator lgbm's best error=0.4303,	best estimator lgbm's best error=0.4303
[flaml.automl: 12-01 18:03:07] {2291} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-01 18:03:07] {2485} INFO -  at 4.2s,	estimator lgbm's best error=0.4303,	best estimator lgbm's best error=0.4303
[flaml.automl: 

[flaml.automl: 12-01 18:03:15] {2291} INFO - iteration 36, current learner xgboost
[flaml.automl: 12-01 18:03:15] {2485} INFO -  at 12.4s,	estimator xgboost's best error=0.2482,	best estimator xgboost's best error=0.2482
[flaml.automl: 12-01 18:03:15] {2291} INFO - iteration 37, current learner xgboost
[flaml.automl: 12-01 18:03:16] {2485} INFO -  at 12.5s,	estimator xgboost's best error=0.2482,	best estimator xgboost's best error=0.2482
[flaml.automl: 12-01 18:03:16] {2291} INFO - iteration 38, current learner xgboost
[flaml.automl: 12-01 18:03:16] {2485} INFO -  at 12.7s,	estimator xgboost's best error=0.2468,	best estimator xgboost's best error=0.2468
[flaml.automl: 12-01 18:03:16] {2291} INFO - iteration 39, current learner xgboost
[flaml.automl: 12-01 18:03:16] {2485} INFO -  at 12.9s,	estimator xgboost's best error=0.2468,	best estimator xgboost's best error=0.2468
[flaml.automl: 12-01 18:03:16] {2291} INFO - iteration 40, current learner xgboost
[flaml.automl: 12-01 18:03:16] {2

[flaml.automl: 12-01 18:03:29] {2485} INFO -  at 26.2s,	estimator xgboost's best error=0.2406,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:29] {2291} INFO - iteration 74, current learner extra_tree
[flaml.automl: 12-01 18:03:30] {2485} INFO -  at 26.5s,	estimator extra_tree's best error=0.2517,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:30] {2291} INFO - iteration 75, current learner lgbm
[flaml.automl: 12-01 18:03:30] {2485} INFO -  at 26.7s,	estimator lgbm's best error=0.2518,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:30] {2291} INFO - iteration 76, current learner lgbm
[flaml.automl: 12-01 18:03:30] {2485} INFO -  at 26.8s,	estimator lgbm's best error=0.2518,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:30] {2291} INFO - iteration 77, current learner extra_tree
[flaml.automl: 12-01 18:03:30] {2485} INFO -  at 27.2s,	estimator extra_tree's best error=0.2517,	best estimator xgboos

[flaml.automl: 12-01 18:03:40] {2291} INFO - iteration 110, current learner xgboost
[flaml.automl: 12-01 18:03:41] {2485} INFO -  at 37.8s,	estimator xgboost's best error=0.2406,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:41] {2291} INFO - iteration 111, current learner lgbm
[flaml.automl: 12-01 18:03:42] {2485} INFO -  at 38.4s,	estimator lgbm's best error=0.2454,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:42] {2291} INFO - iteration 112, current learner xgb_limitdepth
[flaml.automl: 12-01 18:03:42] {2485} INFO -  at 39.2s,	estimator xgb_limitdepth's best error=0.2457,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:42] {2291} INFO - iteration 113, current learner xgboost
[flaml.automl: 12-01 18:03:43] {2485} INFO -  at 39.7s,	estimator xgboost's best error=0.2406,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:03:43] {2291} INFO - iteration 114, current learner xgb_limitdepth
[flaml.automl

[flaml.automl: 12-01 18:04:16] {2291} INFO - iteration 147, current learner lgbm
[flaml.automl: 12-01 18:04:17] {2485} INFO -  at 73.6s,	estimator lgbm's best error=0.2454,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:04:17] {2291} INFO - iteration 148, current learner lgbm
[flaml.automl: 12-01 18:04:17] {2485} INFO -  at 74.3s,	estimator lgbm's best error=0.2454,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:04:17] {2291} INFO - iteration 149, current learner rf
[flaml.automl: 12-01 18:04:18] {2485} INFO -  at 74.7s,	estimator rf's best error=0.2493,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:04:18] {2291} INFO - iteration 150, current learner lgbm
[flaml.automl: 12-01 18:04:18] {2485} INFO -  at 75.4s,	estimator lgbm's best error=0.2454,	best estimator xgboost's best error=0.2406
[flaml.automl: 12-01 18:04:18] {2291} INFO - iteration 151, current learner rf
[flaml.automl: 12-01 18:04:19] {2485} INFO -  at 75.8s,	estim

[flaml.automl: 12-01 18:04:45] {2485} INFO -  at 101.5s,	estimator lgbm's best error=0.2454,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:04:45] {2291} INFO - iteration 185, current learner xgboost
[flaml.automl: 12-01 18:04:45] {2485} INFO -  at 102.0s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:04:45] {2291} INFO - iteration 186, current learner xgboost
[flaml.automl: 12-01 18:04:46] {2485} INFO -  at 102.5s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:04:46] {2291} INFO - iteration 187, current learner rf
[flaml.automl: 12-01 18:04:46] {2485} INFO -  at 102.9s,	estimator rf's best error=0.2493,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:04:46] {2291} INFO - iteration 188, current learner extra_tree
[flaml.automl: 12-01 18:04:47] {2485} INFO -  at 103.9s,	estimator extra_tree's best error=0.2436,	best estimator xgbo

[flaml.automl: 12-01 18:05:31] {2485} INFO -  at 148.3s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:05:31] {2291} INFO - iteration 222, current learner lgbm
[flaml.automl: 12-01 18:05:32] {2485} INFO -  at 149.0s,	estimator lgbm's best error=0.2438,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:05:32] {2291} INFO - iteration 223, current learner extra_tree
[flaml.automl: 12-01 18:05:36] {2485} INFO -  at 152.9s,	estimator extra_tree's best error=0.2436,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:05:36] {2291} INFO - iteration 224, current learner xgboost
[flaml.automl: 12-01 18:05:36] {2485} INFO -  at 153.4s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:05:36] {2291} INFO - iteration 225, current learner lgbm
[flaml.automl: 12-01 18:05:37] {2485} INFO -  at 154.0s,	estimator lgbm's best error=0.2438,	best estimator xgb

[flaml.automl: 12-01 18:06:52] {2485} INFO -  at 228.4s,	estimator lgbm's best error=0.2426,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:06:52] {2291} INFO - iteration 259, current learner extra_tree
[flaml.automl: 12-01 18:06:54] {2485} INFO -  at 230.5s,	estimator extra_tree's best error=0.2420,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:06:54] {2291} INFO - iteration 260, current learner xgboost
[flaml.automl: 12-01 18:06:54] {2485} INFO -  at 231.1s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:06:54] {2291} INFO - iteration 261, current learner xgboost
[flaml.automl: 12-01 18:06:55] {2485} INFO -  at 231.6s,	estimator xgboost's best error=0.2399,	best estimator xgboost's best error=0.2399
[flaml.automl: 12-01 18:06:55] {2291} INFO - iteration 262, current learner xgboost
[flaml.automl: 12-01 18:06:55] {2485} INFO -  at 232.3s,	estimator xgboost's best error=0.2399,	best esti

[flaml.automl: 12-01 18:07:35] {2291} INFO - iteration 296, current learner lgbm
[flaml.automl: 12-01 18:07:35] {2485} INFO -  at 272.1s,	estimator lgbm's best error=0.2397,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:07:35] {2291} INFO - iteration 297, current learner rf
[flaml.automl: 12-01 18:07:36] {2485} INFO -  at 273.3s,	estimator rf's best error=0.2426,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:07:36] {2291} INFO - iteration 298, current learner lgbm
[flaml.automl: 12-01 18:07:37] {2485} INFO -  at 274.0s,	estimator lgbm's best error=0.2397,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:07:37] {2291} INFO - iteration 299, current learner lgbm
[flaml.automl: 12-01 18:07:38] {2485} INFO -  at 274.6s,	estimator lgbm's best error=0.2397,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:07:38] {2291} INFO - iteration 300, current learner lgbm
[flaml.automl: 12-01 18:07:38] {2485} INFO -  at 275.3s,	estimator 

[flaml.automl: 12-01 18:08:21] {2485} INFO -  at 318.2s,	estimator rf's best error=0.2417,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:08:21] {2291} INFO - iteration 335, current learner xgboost
[flaml.automl: 12-01 18:08:22] {2485} INFO -  at 318.8s,	estimator xgboost's best error=0.2399,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:08:22] {2291} INFO - iteration 336, current learner lgbm
[flaml.automl: 12-01 18:08:23] {2485} INFO -  at 319.4s,	estimator lgbm's best error=0.2397,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:08:23] {2291} INFO - iteration 337, current learner xgboost
[flaml.automl: 12-01 18:08:23] {2485} INFO -  at 320.0s,	estimator xgboost's best error=0.2399,	best estimator lgbm's best error=0.2397
[flaml.automl: 12-01 18:08:23] {2291} INFO - iteration 338, current learner lgbm
[flaml.automl: 12-01 18:08:24] {2485} INFO -  at 320.7s,	estimator lgbm's best error=0.2397,	best estimator lgbm's best error=0.2397
[

In [70]:
y_pred = automl.predict(X_test_enc)
predictions = [round(value) for value in y_pred]
class_names = [str(x) for x in automl.classes_]
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

           1       0.64      0.53      0.58      5170
           2       0.74      0.83      0.78     29487
           3       0.74      0.63      0.68     17464

    accuracy                           0.73     52121
   macro avg       0.71      0.66      0.68     52121
weighted avg       0.73      0.73      0.73     52121



In [71]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("F1 Score = {:.4f}".format(f1_score(y_test, predictions, average='micro')))

Accuracy: 73.28%
F1 Score = 0.7328


# Stacking

In [153]:
%%time

if rebalance == 1:
    X_train_enc = X_train_enc_reb.copy()
    y_train = y_resampled.copy()
    X_test_enc = X_test_enc_reb.copy()

rf_param = {'n_estimators': [1000], 
'min_samples_split':[20]}

clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = KNeighborsClassifier(n_neighbors=5)
# clf3 = SVC(kernel='rbf', probability=True)
clf4 = RandomForestClassifier(n_estimators=1000, min_samples_split=20, random_state=0)
clf5 = XGBClassifier()

classifiers=[('DT', clf1), ('RF', clf4)]

sclf = StackingClassifier(
    classifiers=[c[1] for c in classifiers], meta_classifier=XGBClassifier(), fit_base_estimators=True,
    use_probas=True, average_probas=False)

for mdl in classifiers+[('Stacking (XGB)', sclf)]:
    
    clf = mdl[1]
    clf = clf.fit(X_train_enc, y_train)

    y_pred = clf.predict(X_test_enc)
    predictions = [round(value) for value in y_pred]
        
    print(f"{mdl[0]}: Micro-Average F1 = {f1_score(y_test, predictions, average='micro')}")


DT: Micro-Average F1 = 0.5653575334318222
RF: Micro-Average F1 = 0.5484929299130868
[16:39:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Stacking (XGB): Micro-Average F1 = 0.47439611672838206
Wall time: 18min 29s


# Random Forest

In [189]:
%%time
param = {'n_estimators': [ 500, 1000], 
'min_samples_split':[20, 50, 500]}
clfR = RandomForestClassifier()
gd_sr = model_selection.GridSearchCV(estimator=clfR,param_grid=param,scoring='f1_micro',cv=10,n_jobs=-1)
gd_sr.fit(X_train_enc, y_train)

Wall time: 1h 11min 16s


In [190]:
gd_sr.cv_results_

{'mean_fit_time': array([286.50690076, 563.83939595, 261.32530448, 553.84772885,
        258.24207578, 380.25137546]),
 'std_fit_time': array([ 2.02313113,  5.45489387,  2.14482062, 59.90037884, 69.1225982 ,
        35.04474583]),
 'mean_score_time': array([ 6.53273108, 16.93160162,  3.58291526,  7.34754343,  2.45104296,
         4.54105344]),
 'std_score_time': array([1.99962931, 8.29888117, 0.11129861, 0.20768913, 0.09606513,
        0.60652978]),
 'param_min_samples_split': masked_array(data=[20, 20, 50, 50, 500, 500],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[500, 1000, 500, 1000, 500, 1000],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_split': 20, 'n_estimators': 500},
  {'min_samples_split': 20, 'n_estimators': 1000},
  {'min_samples_split': 50, 'n_estimators': 500},
  {'m

In [191]:
print(gd_sr.best_score_)
print(gd_sr.best_params_)

0.7609315042210285
{'min_samples_split': 50, 'n_estimators': 500}


In [138]:
clf = RandomForestClassifier(min_samples_split= 50, n_estimators= 500)
clf = clf.fit(X_train_enc, y_train)

In [150]:
y_pred = clf.predict(X_test_enc)
predictions = [round(value) for value in y_pred]
print(f"Micro-Average F1 = {f1_score(y_test, predictions, average='micro')}")

Micro-Average F1 = 0.5599086740469292


# make prediction on official test data set

In [60]:
model = automl

In [61]:
%%time
enc_train, enc_test = prepare_inputs(X_train, test_values)

left table: (208480, 38), right table: (208480, 79)merged table: (208480, 117)
left table: (86868, 38), right table: (86868, 79)merged table: (86868, 117)
Wall time: 789 ms


In [62]:
y_pred_official = model.predict(enc_test)

In [63]:
y_pred_official

array([3, 2, 3, ..., 2, 2, 1], dtype=int64)

In [64]:
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,has_superstructure_adobe_mud_cs_min,has_superstructure_adobe_mud_cs_max,has_superstructure_adobe_mud_em_min,has_superstructure_adobe_mud_em_max,has_superstructure_mud_mortar_stone_cs_min,has_superstructure_mud_mortar_stone_cs_max,has_superstructure_mud_mortar_stone_em_min,has_superstructure_mud_mortar_stone_em_max,has_superstructure_stone_flag_cs_min,has_superstructure_stone_flag_cs_max,has_superstructure_stone_flag_em_min,has_superstructure_stone_flag_em_max,has_superstructure_cement_mortar_stone_cs_min,has_superstructure_cement_mortar_stone_cs_max,has_superstructure_cement_mortar_stone_em_min,has_superstructure_cement_mortar_stone_em_max,has_superstructure_mud_mortar_brick_cs_min,has_superstructure_mud_mortar_brick_cs_max,has_superstructure_mud_mortar_brick_em_min,has_superstructure_mud_mortar_brick_em_max,has_superstructure_cement_mortar_brick_cs_min,has_superstructure_cement_mortar_brick_cs_max,has_superstructure_cement_mortar_brick_em_min,has_superstructure_cement_mortar_brick_em_max,has_superstructure_timber_cs_min,has_superstructure_timber_cs_max,has_superstructure_timber_em_min,has_superstructure_timber_em_max,has_superstructure_bamboo_cs_min,has_superstructure_bamboo_cs_max,has_superstructure_bamboo_em_min,has_superstructure_bamboo_em_max,has_superstructure_rc_non_engineered_cs_min,has_superstructure_rc_non_engineered_cs_max,has_superstructure_rc_non_engineered_em_min,has_superstructure_rc_non_engineered_em_max,has_superstructure_rc_engineered_cs_min,has_superstructure_rc_engineered_cs_max,has_superstructure_rc_engineered_em_min,has_superstructure_rc_engineered_em_max,geo_level_1_id_prob1,geo_level_1_id_prob2,geo_level_1_id_prob3,geo_level_2_id_prob1,geo_level_2_id_prob2,geo_level_2_id_prob3,geo_level_3_id_prob1,geo_level_3_id_prob2,geo_level_3_id_prob3
0,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013117,0.179184,0.807699,0.008621,0.284483,0.706897,0.071429,0.142857,0.785714
1,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083862,0.666650,0.249488,0.000000,0.831169,0.168831,0.000000,1.000000,0.000000
2,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,2.0,800.0,800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.131600,0.739218,0.129182,0.000000,0.870968,0.129032,0.000000,0.000000,1.000000
3,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,20.0,4200.0,14000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.354230,0.559168,0.086602,0.508870,0.453782,0.037348,0.576108,0

In [65]:
test_values = pd.read_csv("test_values.csv")

In [66]:
submission_file = pd.DataFrame({'building_id':test_values.building_id, 'damage_grade':y_pred_official})
submission_file

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,3
86865,1049160,2
86866,442785,2


In [48]:
submission_file.to_csv('sibmission_file_geo_prob_flaml.csv', index=False)